In [1]:
from edward.models import *
from edward.util import Progbar
import edward as ed
import numpy as np
import tensorflow as tf
from keras.layers import Embedding, Dense

H = 5
D = 2
V = 10
E = 2
batch_size = 10
M = 10
nb_steps = 5

X_train = np.zeros([10000, nb_steps ], dtype=np.int32)
y_train = np.zeros([10000, 1 ], dtype=np.int32)

Using TensorFlow backend.


In [2]:
from neuralfingerprint import load_data

task_params = {'target_name' : 'measured log solubility in mols per litre',
               'data_file'   : 'delaney.csv'}
N_train = 800
N_val   = 20
N_test  = 20

print "Loading data..."
traindata, valdata, testdata = load_data(
    task_params['data_file'], (N_train, N_val, N_test),
    input_name='smiles', target_name=task_params['target_name'])
train_inputs, train_targets = traindata
val_inputs,   val_targets   = valdata
test_inputs,  test_targets  = testdata
print "Loaded data"

Loading data...
Loaded data


In [3]:
# DATA
x_train = train_inputs
x_test = test_inputs

str1 = ''.join(train_inputs)
vocab = list(set(str1));
vocab_size = len(vocab)
encoder = dict(zip(vocab, range(vocab_size)))
decoder = {v: k for k, v in encoder.items()}

In [4]:
x_train[0:10]

array(['OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O ',
       'Cc1occc1C(=O)Nc2ccccc2', 'CC(C)=CCCC(C)=CC(=O)',
       'c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43', 'c1ccsc1', 'c2ccc1scnc1c2 ',
       'Clc1cc(Cl)c(c(Cl)c1)c2c(Cl)cccc2Cl',
       'CC12CCC3C(CCc4cc(O)ccc34)C2CCC1O',
       'ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl',
       'COc5cc4OCC3Oc2c1CC(Oc1ccc2C(=O)C3c4cc5OC)C(C)=C '], dtype='|S98')

In [12]:
encoded_list = []
for i in x_train[0]: encoded_list.append(encoder[i])

In [26]:
el = np.array(encoded_list)
el2 = el.reshape(1,len(el))
print(el2)


array([[19, 14, 14,  6, 19, 14,  3, 19, 14, 14,  7, 19, 14,  3, 19, 14,
         3, 14,  1, 20,  2, 26,  5, 26, 26, 26, 26, 26,  5,  2, 14,  3,
        19,  2, 14,  3, 19,  2, 14,  7, 19,  2, 14,  3, 19,  2, 14,  3,
        19,  2, 14,  6, 19,  0]])

In [5]:
def lstm_cell(x, h, c, wx, wh, b, name=None, reuse=False):
  """LSTM returning hidden state and content cell at a specific timestep."""
  nin = x.shape[-1].value
  nout = h.shape[-1].value
  z = tf.matmul(x, wx) + tf.matmul(h, wh) + b
  i, f, o, u = tf.split(z, 4, axis=1)
  i = tf.sigmoid(i)
  f = tf.sigmoid(f + 1.0)
  o = tf.sigmoid(o)
  u = tf.tanh(u)
  c = f * c + i * u
  h = o * tf.tanh(c)
  return h, c

In [11]:
x = tf.placeholder(tf.int32, [batch_size, nb_steps ], name='x')    
h = tf.fill(tf.stack([tf.shape(x)[0], hidden_size]), 0.0)
c = tf.fill(tf.stack([tf.shape(x)[0], hidden_size]), 0.0)

nin = x.shape[-1].value
nout = h.shape[-1].value
wx = tf.get_variable("kernel/input", [nin, nout * 4],
                         dtype=tf.float32,
                         initializer=tf.orthogonal_initializer(1.0))
wh = tf.get_variable("kernel/hidden", [nout, nout * 4],
                         dtype=tf.float32,
                         initializer=tf.orthogonal_initializer(1.0))
b = tf.get_variable("bias", [nout * 4],
                        dtype=tf.float32,
                        initializer=tf.constant_initializer(0.0))



In [23]:
y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [21]:
timesteps = 64
scale_out = 1

def RNN(input,timesteps,hidden_size,scale_out,wx,wh,b):
    h = tf.fill(tf.stack([tf.shape(input)[0], hidden_size]), 0.0)
    c = tf.fill(tf.stack([tf.shape(input)[0], hidden_size]), 0.0)
    hs = []
    reuse = None
    for t in range(timesteps):
        if t > 0:
            reuse = True
        xt = input[:, t, :]
        h, c = lstm_cell(xt, h, c,wx,wh,b, name="lstm", reuse=reuse)
        theloc=tf.layers.dense(h,1, name="dense") ###I use the latest hidden values to calculate output

    #######problem code#############
    #output=theloc
    thescale=tf.fill(tf.stack([tf.shape(input)[0], 1]), scale_out)
    output=Normal(loc=theloc,scale=thescale )
    ###################################
    return output

#######Run the inference with the following interface########################
Y=RNN(x,timesteps,hidden_size,scale_out,wx,wh,b)
inference = ed.KLqp({wx: qwx, wh:qwh,b: qb}, data={x: X_train, Y: y_train})
inference.run(n_samples=5, n_iter=20)



ValueError: Index out of range using input dim 2; input has only 2 dims for 'strided_slice_22' (op: 'StridedSlice') with input shapes: [10,5], [3], [3], [3] and with computed input tensors: input[3] = <1 1 1>.